# Problem Set 9

## Example: predicting colon cancer from stool microbiome composition

In [ ]:
# plot settings
options(repr.plot.width=15, repr.plot.height=10)

In [ ]:
# Install a package BioConductor ExperimentHub to access the example data
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install()
BiocManager::install("ExperimentHub")

# Install glmnet for LASSO and Elastic Net regression
install.packages("glmnet")

In [ ]:

library("BiocManager")
library("glmnet")


In [ ]:
library("ExperimentHub")

eh = ExperimentHub()
data = eh[["EH361"]]


## 1. Explore the data set

Explore our data set. The rows are the presence of microbes in the gut. They also contain the presence of cancer. The columns represent different patients. 

__i) How many patients are in the data set?__

__ii) How many species of microbe are in the data set?__

There are possible 3 disease states. Create a data set with only patients who are either "N" (No cancer) or "cancer" (cancer). In other words, remove the patiends with adenomas.

__iii) After removing the patients with adenomas, how many patients are in the data set?__

In [ ]:
data

In [ ]:
rownames(data)

In [ ]:
colnames(data)

For simplicity, let's only use the "n" and "cancer" states (and remove the adenomas).


In [ ]:
data$disease

dataCancerNoCancer = data[, data$disease %in% c("n", "cancer")]

dataCancerNoCancer$disease

## 2. Lasso regression
Let's perform lasso regression. In this case, the response variable is categorical (cancer or no cancer) so we can use a binomial model, which is a subset of logistic models. 

__i) According to the crossvalidation analysis, how many species of microbe should we include in a predictive model of colon cancer?__

In [ ]:
y = factor(dataCancerNoCancer$disease)
x = t(exprs(dataCancerNoCancer))

lassoFit = glmnet(y=y, x=x, family="binomial")

plot(lassoFit, xvar = "lambda", label = TRUE)


In [ ]:
crossValidationOutput <- cv.glmnet(y=factor(dataCancerNoCancer$disease),
                                   x=t(exprs(dataCancerNoCancer)), family="binomial")

plot(crossValidationOutput)

In [ ]:
bestLambda = crossValidationOutput$lambda.min
confusionMatrix = predict(lassoFit, newx = t(exprs(dataCancerNoCancer)), type="class",s=bestLambda)
table(confusionMatrix, dataCancerNoCancer$disease)

ii) Write code to extract the number of false positives (non-cancer patients that are predicted to have cancer) and false negatives (cancer patients that are predicted to not have cancer).

## 3.  Elastic Net regression

The characteristic feature of Ridge regression is the penalty

$$\mbox{log}\left(L(\beta)\right) - \lambda \sum_i  \beta_i ^2,$$

while the penalty for Lasso regression is

$$\mbox{log}\left(L(\beta)\right) - \lambda \sum_i | \beta_i |.$$

In this Problem Set, we explore the penalty

$$\mbox{log}\left(L(\beta)\right) - \lambda \left(\alpha \sum_i | \beta_i | +  (1-\alpha) \sum_i  \beta_i ^2\right),$$

which is called __Elastic Net__.  

i) In parameter space, Ridge Regression corresponds to finding optimal parameters on a circle, while LASSO regression corresponds to finding optimal parameters on a diamond. What shape does Elastic Net correspond to?

ii). The `glmnet` package was built for Elastic Net regression. Look up the [glmnet package help files](https://www.rdocumentation.org/packages/glmnet/versions/3.0-2/topics/glmnet) to find out how to perform Elastic Net regression for a specific $\alpha$. Do this for $\alpha=0.5$, and return the confusion matrix.

iii) Perform a sweep over $\alpha=0$ to $\alpha=1$. Plot the number of species included versus $\alpha$.

Hint: The cv.glmnet output object has a returns a value `$nzero`, which is the number of nonzero factors, which is the number of species desired.  

iv) What $\alpha$ value minimizes the number of false positives (non-cancer patients that are predicted to have cancer)? What $\alpha$ value minimizes the number of false negatives (cancer patients that are predicted to not have cancer)?